# PARAMETERS

In [ ]:
####### CONFIGURATION

class CFG:

    # environment
    environment = 'local'  # work environment ('kaggle', 'colab', 'local')
    device      = 'GPU'    # device ('CPU', 'GPU', 'TPU')
    num_workers = 4        # no. cores

    # general
    version  = 131   # notebook version (for saving outputs)
    debug    = False  # debug mode runs 5 batches for 2 epochs 
    training = True  # if False, only inference is run
    seed     = 13353 # random state

    # data
    num_folds  = 5      # number of CV folds
    data_2019  = True   # append 2019 labeled data to training folds (1a)
    data_pl    = 0.2    # False or percentage of appended pseudo-labeled data (1b)
    data_ext   = False  # False or list with external dataset ids (2, 3, 4, 5)
    drop_dupl  = True   # drop duplicate images from training folds
    drop_outs  = False  # drop outliers from training folds
    oversample = False  # enable oversampling through WeightedRandomSampler()
    
    # label noise
    drop_noise  = False  # False or percentage of removed noisy labels in training folds
    flip_noise  = False  # False or percentage of flipped noisy labels in training folds

    # image size and augmentations 
    image_size   = 384               # image size after random crop
    crop_scale   = (0.10, 1)          # min scale, max scale
    gr_shuffle   = (3, 3)             # number of tiles in shuffled grid
    ssr          = [0.05, 0.05, 360]  # shift, scale, rotation limits
    huesat       = [20, 20, 20]       # hue, saturation, value limits
    bricon       = [0.1, 0.1]         # brightness, contrast limits
    blur_limit   = 3                  # blur limit
    dist_limit   = 0.1                # distortion limit
    cutout       = [5, 0.1]           # number of squares, size of squares
    clahe        = [0, 1]             # clip limit, size of grid
    p_augment    = 0.5                # prob. of augmentations except for flips
    cutmix       = [0, 1]             # cutmix batch-level probability, alpha
    normalize    = False              # pixel normalization (False, 'dataset', 'imagenet')
    pairing      = False              # overlay of a random image from the same class
    pairing_prob = 0.6                # probability of an image to get another image added
    pixel_crop   = 150                # hight and width of the crop

    # model architecture
    backbone  = 'vit_base_patch16_384'  # convolutional backbone
    weights   = 'imagenet'              # weights ('empty', 'imagenet', 'custom')
    attention = False                   # whether to include attention module (efficientnet only)
    save_all  = False                   # save weights from each epoch

    # pretrained model
    pr_version     = 3   # notebook version (2, 3, 4, 5)
    pr_num_classes = 10  # no. classes (2: 4, 3: 10, 4: 2, 5: 17)

    # training
    batch_size  = 16    # no. images per batch
    num_epochs  = 10    # no. epochs per fold
    fine_tune   = 2     # fine-tuning dense layers (False or no. epochs)
    accum_iter  = 2     # no. batches for gradient accumalation
    use_amp     = True  # automated mixed precision mode

    # learning rate and optimizer
    eta     = 1e-4    # starting learning rate
    eta_min = 1e-6    # ending learning rate
    optim   = 'AdamP' # LR optimizer ('Adam', 'AdamW', 'AdamP')
    decay   = 0       # weight decay of optimizer (L2 regularization)

    # learning rate scheduler
    warmup          = 1                  # no. epochs for warmup
    warmup_freeze   = False              # freeze deep layers during warmup
    schedule        = 'CosineAnnealing'  # LR scheduler after warmup
    update_on_batch = True               # update LR after every batch or epoch

    # loss function
    loss_fn     = 'OHEM'         # loss function ('CE', 'OHEM', 'SCE', 'CCE', 'Focal', 'FocalCosine', 'Taylor', 'BiTempered')
    smoothing   = 0.2          # label smoothing (works with all losses)
    ohem        = 0.8          # OHEM loss parameters: top-k percentage
    sce         = [0.1, 1.0]   # SCE loss parameters: alpha, beta
    cce         = 5            # CCE loss parameters: gamma
    focal       = [1, 2]       # Focal loss parameters: alpha, gamma
    focalcosine = [1, 2, 0.1]  # FocalCosine loss parameters: alpha, gamma, xent
    taylor      = 2            # Taylor loss parameters: n
    bitempered  = [0.3, 1.0]   # BiTempered loss parameters: t1, t2

    # epoch-based changes
    step_size  = False  # False or list with image_size multiplier for each epoch
    step_class = False  # False or list with num_classes for each epoch (2 or 5)
    step_p_aug = False  # False or list with p_augment multiplier for each epoch
    step_loss  = False  # False or list with loss functions for each epoch

    # inference
    num_tta = 1  # no. TTA flips (between 1 and 8)

In [ ]:
####### CONVERT CONFIGURATION

CFG = dict(vars(CFG))
for key in ['__dict__', '__doc__', '__module__', '__weakref__']:
    del CFG[key]

# PREPARATIONS

In [ ]:
####### ENVIRONMENT SETUP

##### COLAB

import os
if (CFG['environment'] == 'colab') and (not os.path.exists('/content/cassava/')):

    # mount drive
    from google.colab import drive
    drive.mount('/content/drive')  

    # set up Kaggle API
    !pip install --upgrade --force-reinstall --no-deps -q kaggle
    !mkdir -p ~/.kaggle
    !cp '/content/drive/MyDrive/Competitions/kaggle.json' ~/.kaggle/
    !ls ~/.kaggle

    # download data
    !mkdir '/content/cassava/'
    !kaggle kernels output kozodoi/merge-and-zip-2019-2020-data -p '/content/cassava/'
    !unzip -o -q '/content/cassava/images.zip' -d '/content/cassava/'
    !rm -rf '/content/cassava/images.zip'
    
    
##### KAGGLE

if (CFG['environment'] == 'kaggle') and (not os.path.exists('cassava/')):

    # extract data
    !mkdir 'cassava/'
    !unzip -o -q '/kaggle/input/merge-and-zip-2019-2020-data/images.zip' -d 'cassava/'
    !cp '/kaggle/input/merge-and-zip-2019-2020-data/train.csv' 'cassava/'

In [ ]:
####### WORKING PATHS

# kaggle
if CFG['environment'] == 'kaggle':
    CFG['data_path']  = 'cassava/'
    CFG['model_path'] = ''
    CFG['out_path']   = ''

# colab
if CFG['environment'] == 'colab':
    CFG['data_path']  = '/content/cassava/'
    CFG['model_path'] = '/content/drive/MyDrive/Cassava/pretraining/v' + str(CFG['pr_version']) + '/'
    CFG['out_path']   = '/content/drive/MyDrive/Cassava/output/v'      + str(CFG['version'])    + '/'
    if not os.path.exists(CFG['out_path']):
        os.mkdir(CFG['out_path'])

# local
if CFG['environment'] == 'local':
    CFG['data_path']  = '../../data/'
    CFG['model_path'] = '../../pretraining/v' + str(CFG['pr_version']) + '/'
    CFG['out_path']   = '../../output/v'      + str(CFG['version'])    + '/'
    if not os.path.exists(CFG['out_path']):
        os.mkdir(CFG['out_path'])

In [ ]:
####### PACKAGES FOR TPU

if CFG['device'] == 'TPU':
        
    # install XLA
    xla_version = 'nightly' # (1.7, 'nightly')
    !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
    !python pytorch-xla-env-setup.py --version $xla_version

    # settings
    if CFG['environment'] == 'colab':
        import os
        assert os.environ['COLAB_TPU_ADDR']
    os.environ['XLA_USE_BF_16'] = '1'
    os.environ['XLA_TENSOR_ALLOCATOR_MAXSIZE'] = '100000000'

    # imports
    import torch_xla
    import torch_xla.core.xla_model as xm
    import torch_xla.distributed.xla_multiprocessing as xmp
    import torch_xla.distributed.parallel_loader as pl

In [ ]:
####### PACKAGES

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import RandomSampler, SequentialSampler, WeightedRandomSampler
from torch.utils.data.distributed import DistributedSampler
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, ReduceLROnPlateau
from torch.autograd import Function

!pip install timm
import timm
from timm.utils import *

from contextlib import suppress

!pip install --upgrade -U albumentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

!pip install git+'https://github.com/ildoonet/pytorch-gradual-warmup-lr.git'
from warmup_scheduler import GradualWarmupScheduler  

!pip install adamp
from adamp import AdamP

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import cv2

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold
from scipy.special import softmax

from tqdm import tqdm

import random
import math
import time
import sys
import os
import pickle

import gc

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
####### HELPER FUNCTIONS

sys.path.append('../../codes')

# utilities
from utilities import *

# data processing
from data import LeafData, get_data
from augmentations import get_augs, rand_bbox, cutmix_fn, get_tta_flips

# losses
from losses import *

# model
from model import *

# training and inference
from train_epoch import train_epoch
from valid_epoch import valid_epoch
from run_fold import run_fold
from plot_results import plot_results

In [ ]:
####### TRAINING DEVICE

if CFG['device'] == 'TPU':
    print('Training on TPU...')
    device = torch.device('cpu')
    
if CFG['device'] == 'GPU':
    print('Training on GPU...')
    if CFG['environment'] == 'local':
        device = torch.device('cuda:1')
    else:
        device = torch.device('cuda:0')

if CFG['device'] == 'CPU':
    print('Training on CPU...')
    device = torch.device('cpu') 

# DATA PREP

In [ ]:
####### 2020 COMPETITION DATA

# import data
df = pd.read_csv(CFG['data_path'] + 'train.csv')
df = df.loc[df['source'] == 2020].reset_index(drop = True)
    
# num classes
CFG['num_classes'] = df['label'].nunique()

# partitioning
df['fold'] = -1
skf = StratifiedKFold(n_splits = CFG['num_folds'], random_state = CFG['seed'], shuffle = True)
for fold, (trn_idx, val_idx) in enumerate(skf.split(df, df['label'])):
    df.loc[val_idx, 'fold'] = fold
    
# check folds
pd.crosstab(df['fold'], df['label'])

In [ ]:
####### 2019 COMPETITION DATA

##### LABELED DATA

if CFG['data_2019']:

    # import data
    df_2019 = pd.read_csv(CFG['data_path'] + 'train.csv')
    df_2019 = df_2019.loc[df_2019['source'] == 2019].reset_index(drop = True)
    df_2019['fold'] = -1
        
    # check classes
    display(df_2019['label'].value_counts())


##### PSEUDO-LABELED DATA

if CFG['data_pl']:

    # import
    if CFG['environment'] == 'colab':
        !kaggle kernels output kozodoi/pseudo-labeling-v57 -p '/content/cassava/'
        !unzip -o -q '/content/cassava/unlabeled_images.zip' -d '/content/cassava/train_images/'
        !rm -rf '/content/cassava/unlabeled_images.zip'
    df_pl = pd.read_csv(CFG['data_path'] + 'df_pl.csv')
    df_pl['fold'] = -1

    # filter confident predictions
    df_pl['confidence'] = df_pl.filter(like = 'c').max(axis = 1)
    df_pl = df_pl.sort_values('confidence', ascending = False)
    df_pl = df_pl.head(int(CFG['data_pl'] * len(df_pl)))
    df_pl = df_pl[['image_id', 'label', 'fold']]

    # check classes
    display(df_pl['label'].value_counts())

In [ ]:
####### EXTERNAL DATA [HEALTHY]

if CFG['data_ext']:

    # placeholder
    df_ext = pd.DataFrame(columns = ['image_id', 'label', 'source'])


    ##### DATASET 2

    if 2 in CFG['data_ext']:

        # import images
        !kaggle kernels output kozodoi/prepare-dataset-2 -p '/content/cassava/'
        !unzip -o -q '/content/cassava/images.zip' -d '/content/cassava/train_images/'
        !rm -rf '/content/cassava/images.zip'

        # filtering
        df_2 = pd.read_csv(CFG['data_path'] + 'df_2.csv')
        df_2 = df_2.loc[df_2['label'] == 4].reset_index(drop = True)
        df_2['source'] = 2
        df_ext = pd.concat([df_ext, df_2], axis = 0).reset_index(drop = True)
        

    ##### DATASET 3

    if 3 in CFG['data_ext']:

        # import
        !kaggle kernels output kozodoi/prepare-dataset-3 -p '/content/cassava/'
        !unzip -o -q '/content/cassava/images.zip' -d '/content/cassava/train_images/'
        !rm -rf '/content/cassava/images.zip'

        # filtering
        df_3 = pd.read_csv(CFG['data_path'] + 'df_3.csv')
        df_3 = df_3.loc[df_3['label'] == 4].reset_index(drop = True)
        df_3['source'] = 3
        df_ext = pd.concat([df_ext, df_3], axis = 0).reset_index(drop = True)


    ##### DATASET 4

    if 4 in CFG['data_ext']:

        # import
        !kaggle kernels output kozodoi/prepare-dataset-4 -p '/content/cassava/'
        !unzip -o -q '/content/cassava/images.zip' -d '/content/cassava/train_images/'
        !rm -rf '/content/cassava/images.zip'

        # filtering
        df_4 = pd.read_csv(CFG['data_path'] + 'df_4.csv')
        df_4 = df_4.loc[df_4['label'] == 4].reset_index(drop = True)
        df_4['source'] = 4
        df_ext = pd.concat([df_ext, df_4], axis = 0).reset_index(drop = True)


    ##### DATASET 5

    if 5 in CFG['data_ext']:

        # import
        !kaggle kernels output kozodoi/prepare-dataset-5 -p '/content/cassava/'
        !unzip -o -q '/content/cassava/images.zip' -d '/content/cassava/train_images/'
        !rm -rf '/content/cassava/images.zip'

        # filtering
        df_5 = pd.read_csv(CFG['data_path'] + 'df_5.csv')
        df_5 = df_5.loc[df_5['label'] == 4].reset_index(drop = True)
        df_5['source'] = 5
        df_ext = pd.concat([df_ext, df_5], axis = 0).reset_index(drop = True)


    ##### PARTITIONING

    df_ext['source'] = df_ext['source'].astype('int')
    df_ext['fold']   = -1
    display(df_ext['source'].value_counts())

In [ ]:
####### IDENTIFY BAD EXAMPLES

##### DUPLICATES

if CFG['drop_dupl']:

    # duplicates: all data with DBSCAN
    duplicates   = [['extra-image-1540.jpg', '3286430972.jpg'], ['extra-image-5393.jpg', '1007196516.jpg'], ['extra-image-9233.jpg', '177429020.jpg'], ['extra-image-11388.jpg', '401103417.jpg'], ['extra-image-4308.jpg', '2293657925.jpg'], ['train-cbsd-216.jpg', 'train-cbb-381.jpg', 'test-img-1450.jpg'], ['train-cgm-90.jpg', 'train-cbsd-791.jpg', 'test-img-2051.jpg'], ['extra-image-1924.jpg', '1710187645.jpg'], ['extra-image-5502.jpg', '3513881408.jpg'], ['extra-image-2527.jpg', '4080972605.jpg'], ['extra-image-5475.jpg', '2309908274.jpg'], ['extra-image-428.jpg', '490262929.jpg'], ['train-cgm-373.jpg', 'test-img-3468.jpg'], ['train-cbsd-123.jpg', 'test-img-1610.jpg'], ['extra-image-549.jpg', '2259763259.jpg'], ['extra-image-6922.jpg', 'extra-image-12476.jpg', '825340900.jpg'], ['extra-image-2618.jpg', '1318419572.jpg'], ['extra-image-7208.jpg', 'extra-image-109.jpg', '1273363177.jpg'], ['extra-image-11114.jpg', '1618418212.jpg'], ['test-img-1455.jpg', '4016154109.jpg'], ['extra-image-563.jpg', '3789300725.jpg'], ['train-cmd-2506.jpg', 'train-cmd-1997.jpg', 'test-img-1792.jpg'], ['extra-image-15978.jpg', '1720773250.jpg'], ['train-cmd-366.jpg', 'test-img-3080.jpg'], ['extra-image-2732.jpg', '490603548.jpg'], ['extra-image-14790.jpg', 'extra-image-12199.jpg'], ['train-cmd-900.jpg', '2651363651.jpg'], ['extra-image-933.jpg', '479472063.jpg'], ['extra-image-6992.jpg', '3676057452.jpg'], ['532464272.jpg', '3361673129.jpg'], ['extra-image-10457.jpg', '3118293114.jpg'], ['extra-image-633.jpg', '3069603925.jpg'], ['train-cgm-563.jpg', '4110644267.jpg'], ['extra-image-1372.jpg', '1466391339.jpg'], ['train-cbb-282.jpg', '3476081387.jpg'], ['extra-image-4289.jpg', '2333536768.jpg'], ['extra-image-9788.jpg', '1558894441.jpg'], ['extra-image-9728.jpg', '2704051934.jpg'], ['3507045403.jpg', '2243541656.jpg'], ['extra-image-9779.jpg', '1719304067.jpg'], ['extra-image-3139.jpg', '3361179403.jpg'], ['extra-image-15471.jpg', '1621255319.jpg'], ['extra-image-11326.jpg', '3647124621.jpg'], ['extra-image-6559.jpg', '2519536403.jpg'], ['extra-image-14943.jpg', '952303505.jpg'], ['train-cbsd-1150.jpg', '867127390.jpg'], ['extra-image-5923.jpg', '1091727259.jpg'], ['train-cmd-2170.jpg', '3903538298.jpg'], ['extra-image-12924.jpg', '1131959133.jpg'], ['extra-image-13477.jpg', '2904848270.jpg'], ['extra-image-3871.jpg', '4059169921.jpg'], ['extra-image-4513.jpg', '3662840429.jpg'], ['extra-image-4605.jpg', '232710623.jpg'], ['extra-image-9217.jpg', '1200108798.jpg'], ['extra-image-2419.jpg', '690440568.jpg', '3803823261.jpg'], ['extra-image-12887.jpg', '1989789271.jpg'], ['3251960666.jpg', '2925605732.jpg', '1770746162.jpg'], ['extra-image-4112.jpg', '800250438.jpg'], ['extra-image-10326.jpg', '2888967355.jpg'], ['train-cmd-2615.jpg', 'test-img-3118.jpg'], ['test-img-2592.jpg', '943110824.jpg'], ['extra-image-9616.jpg', '1332855741.jpg'], ['test-img-998.jpg', '58720038.jpg'], ['extra-image-5076.jpg', '2801552324.jpg'], ['extra-image-9334.jpg', '4245562546.jpg'], ['extra-image-12355.jpg', '185236448.jpg'], ['extra-image-5576.jpg', '1761795923.jpg'], ['extra-image-7798.jpg', '3148313410.jpg'], ['train-cgm-8.jpg', '3859028489.jpg'], ['extra-image-13482.jpg', '426528709.jpg'], ['extra-image-12337.jpg', '4089645917.jpg'], ['extra-image-6633.jpg', '2437547294.jpg'], ['extra-image-2857.jpg', '2246452498.jpg'], ['extra-image-590.jpg', '3677086623.jpg'], ['extra-image-10042.jpg', '169189292.jpg'], ['extra-image-8920.jpg', 'extra-image-2601.jpg'], ['extra-image-4990.jpg', '4138769021.jpg'], ['train-cbsd-1437.jpg', 'test-img-3607.jpg', 'test-img-2634.jpg'], ['train-cmd-2459.jpg', 'test-img-698.jpg', 'test-img-3742.jpg', 'test-img-2354.jpg'], ['test-img-1194.jpg', '1007700625.jpg'], ['extra-image-5652.jpg', '3418761424.jpg'], ['extra-image-12886.jpg', '1529177483.jpg'], ['588169966.jpg', '1733354827.jpg'], ['train-cmd-2102.jpg', 'train-cmd-2045.jpg', 'train-cmd-1076.jpg', 'test-img-183.jpg'], ['test-img-3037.jpg', '2760272941.jpg'], ['extra-image-6996.jpg', '3258193135.jpg'], ['extra-image-10309.jpg', '19872828.jpg', '1796762442.jpg'], ['train-cmd-790.jpg', 'train-cmd-2355.jpg'], ['extra-image-9742.jpg', '3132128369.jpg'], ['extra-image-7929.jpg', '2828723686.jpg'], ['extra-image-14835.jpg', '744619434.jpg'], ['extra-image-8184.jpg', '3198115498.jpg'], ['extra-image-15188.jpg', '239700501.jpg'], ['extra-image-3641.jpg', '1694968228.jpg'], ['extra-image-4976.jpg', '1660551148.jpg'], ['test-img-1767.jpg', 'test-img-1219.jpg'], ['extra-image-2468.jpg', '4195598174.jpg'], ['extra-image-12556.jpg', '3889376143.jpg'], ['extra-image-1563.jpg', '1803055979.jpg'], ['extra-image-2068.jpg', '1089759212.jpg'], ['extra-image-8657.jpg', '3055297068.jpg'], ['extra-image-5237.jpg', '245076718.jpg'], ['extra-image-11168.jpg', '3354494955.jpg'], ['extra-image-12603.jpg', '4134659314.jpg'], ['extra-image-16033.jpg', '597389720.jpg'], ['extra-image-14012.jpg', '4234497061.jpg'], ['test-img-138.jpg', '3884189662.jpg'], ['extra-image-1992.jpg', '1982702143.jpg'], ['extra-image-5798.jpg', '3048105792.jpg'], ['test-img-439.jpg', '2259667328.jpg'], ['test-img-1145.jpg', '1356659344.jpg'], ['extra-image-9462.jpg', '3826670209.jpg'], ['extra-image-456.jpg', '945966296.jpg'], ['train-cbb-129.jpg', '519050764.jpg'], ['extra-image-12241.jpg', '2876852928.jpg'], ['extra-image-13234.jpg', '1973109559.jpg'], ['extra-image-6232.jpg', '2659874728.jpg'], ['extra-image-3883.jpg', '2086525919.jpg'], ['extra-image-12735.jpg', '1263625799.jpg'], ['extra-image-2704.jpg', 'extra-image-12619.jpg', '534364866.jpg'], ['extra-image-7008.jpg', '459566440.jpg'], ['extra-image-771.jpg', '1893842412.jpg'], ['extra-image-4438.jpg', '3036253841.jpg'], ['extra-image-6310.jpg', '1663857014.jpg'], ['extra-image-10771.jpg', '1435171668.jpg'], ['extra-image-6392.jpg', '2305895487.jpg'], ['extra-image-8649.jpg', '1831389246.jpg'], ['extra-image-11953.jpg', '1043857809.jpg'], ['test-img-242.jpg', 'test-img-2340.jpg'], ['extra-image-2434.jpg', '234486821.jpg'], ['extra-image-11518.jpg', '1465774459.jpg'], ['extra-image-2589.jpg', '671675016.jpg'], ['extra-image-11209.jpg', '3051541520.jpg'], ['extra-image-7346.jpg', '4070631340.jpg'], ['extra-image-652.jpg', '3746679490.jpg'], ['extra-image-14165.jpg', '2008776850.jpg'], ['extra-image-8273.jpg', '2181262010.jpg'], ['extra-image-6930.jpg', '3649693860.jpg'], ['test-img-3350.jpg', '3709602808.jpg'], ['extra-image-14435.jpg', '2976631036.jpg'], ['extra-image-1152.jpg', '3304610004.jpg'], ['extra-image-4709.jpg', '2810787386.jpg'], ['train-cmd-1507.jpg', 'test-img-33.jpg', 'test-img-235.jpg'], ['extra-image-15422.jpg', '1381973639.jpg'], ['extra-image-2259.jpg', '1061187521.jpg'], ['extra-image-9075.jpg', '2356938780.jpg'], ['extra-image-4587.jpg', '1160018036.jpg'], ['extra-image-14543.jpg', '138109251.jpg'], ['extra-image-9167.jpg', '2583894106.jpg'], ['extra-image-4718.jpg', '1297475127.jpg'], ['extra-image-14501.jpg', '2435784137.jpg'], ['extra-image-13365.jpg', '2126921679.jpg'], ['extra-image-3186.jpg', '324248837.jpg'], ['extra-image-3175.jpg', '1820923810.jpg'], ['421035788.jpg', '1008244905.jpg'], ['extra-image-8581.jpg', '4153074724.jpg'], ['extra-image-5561.jpg', '408051106.jpg'], ['extra-image-6928.jpg', '2396550881.jpg'], ['extra-image-9250.jpg', '3407709856.jpg'], ['train-cbsd-251.jpg', '3001382345.jpg'], ['extra-image-15574.jpg', '726377415.jpg']]
    false_images = ['train-cmd-2506.jpg', 'extra-image-14790.jpg', '53246272.jpg', '3507045403.jpg', 'test-img-3607.jpg', '19872828.jpg', 'extra-image-2619.jpg']
    
    # convert to duplicate list
    list_dupl = []
    for row in duplicates:
        list_dupl += row[:-1]
    list_dupl = list(set(list_dupl))
    list_dupl = [l for l in list_dupl if l not in false_images]
    del duplicates
    print('No. duplicate images:', len(list_dupl))


##### OUTLIERS

if CFG['drop_outs']:

    list_outs = ['1033403106.jpg', '104535906.jpg', '1084966463.jpg', '1200108798.jpg', '1230982659.jpg', '1234375577.jpg', '1329083657.jpg', '133263895.jpg', '1395513159.jpg', '1541808301.jpg', '1576487452.jpg', '1635136764.jpg', '1689108113.jpg', '1765374655.jpg', 
                 '1841279687.jpg', '1870791731.jpg', '1894675387.jpg', '2052899282.jpg', '2151050324.jpg', '2298308938.jpg', '2321669192.jpg', '2378878506.jpg', '2504430507.jpg', '2642216511.jpg', '2666462236.jpg', '2719114674.jpg', '2739645903.jpg', '2786559298.jpg', 
                 '2853279464.jpg', '2889965946.jpg', '2967206024.jpg', '2968477250.jpg', '3002089936.jpg', '3212523761.jpg', '3238801760.jpg', '3250352507.jpg', '3267697264.jpg', '3433580973.jpg', '3806787164.jpg', '4196928486.jpg', '4250554784.jpg', '456647345.jpg', 
                 '479609263.jpg', '511494516.jpg', '534270890.jpg', '587829607.jpg', '65139094.jpg', '690440568.jpg', '74966012.jpg', '854773586.jpg', 'train-cmd-328.jpg', 'train-cbsd-319.jpg', 'train-cgm-223.jpg', 'train-cgm-190.jpg', 'train-cmd-1479.jpg']
    print('No. outlier images:', len(list_outs))


##### NOISY IMAGES

if (CFG['drop_noise']) or (CFG['flip_noise']):

    # import
    df_no = pd.read_csv(CFG['data_path'] + 'oof_median.csv')

    # filter confident predictions
    df_no['pred']      = np.argmax(df_no[['c0', 'c1', 'c2', 'c3', 'c4']].values, axis = 1)
    df_no['confidence'] = df_no[['c0', 'c1', 'c2', 'c3', 'c4']].max(axis = 1)
    df_no = df_no.loc[df_no['pred'] != df_no['label']].reset_index(drop = True)

    # extract top wrong preds per class
    df_no = df_no.sort_values(['label', 'confidence'], ascending = False)
    top_p = CFG['droop_noise'] if CFG['drop_noise'] else CFG['flip_noise']
    df_no = df_no.groupby('label').apply(lambda x: x.head(int(len(x)*top_p))).reset_index(drop = True)
    print('No. noisy images:', len(df_no))

In [ ]:
####### DATASET

class LeafData(Dataset):
    
    # initialization
    def __init__(self, 
                 data, 
                 directory, 
                 transform    = None, 
                 labeled      = False,
                 pairing      = False,
                 pairing_prob = 0.01,
                 pixel_crop   = 50,
                 seed         = 0):
        
        self.data         = data
        self.directory    = directory
        self.transform    = transform
        self.labeled      = labeled
        self.pairing      = pairing
        self.pairing_prob = pairing_prob
        self.seed         = seed
        self.counter      = 0
        self.pixel_crop   = pixel_crop
        if pairing:
            self.sampled_idx = self.data.sample(frac=self.pairing_prob, random_state=self.seed).index
        
    # length
    def __len__(self):
        return len(self.data)
    
    # get item  
    def __getitem__(self, idx):
        
        # import
        path  = os.path.join(self.directory, self.data.iloc[idx]['image_id'])
        image = cv2.imread(path)
        if image is None:
            raise FileNotFoundError(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
        # augmentations
        if self.transform is not None:
            image = self.transform(image = image)['image']
            
        if self.pairing:
            if idx in self.sampled_idx:
                
                try:
                    add_path     = os.path.join(self.directory, self.data.loc[self.data.label==self.data.iloc[idx]['label']].sample(1, random_state=self.seed)['image_id'].iloc[0])
                except:
                    print(self.data.sample(1, random_state=self.seed))
                    add_path     = os.path.join(self.directory, self.data.sample(1, random_state=self.seed)['image_id'].iloc[0])
                    self.counter += 1
                    
                add_image = cv2.imread(add_path)
                add_image = cv2.cvtColor(add_image, cv2.COLOR_BGR2RGB)
                if self.transform is not None:
                    add_image = self.transform(image = add_image)['image']
                    
                max_x = add_image.shape[2] - self.pixel_crop
                max_y = add_image.shape[1] - self.pixel_crop
                
                x = torch.randint(0, max_x, (1,))
                y = torch.randint(0, max_y, (1,))
                crop  = add_image[:, y: y + self.pixel_crop, x: x + self.pixel_crop]
                image[:, y: y + self.pixel_crop, x: x + self.pixel_crop] = crop
                
        
        # output
        if self.labeled:
            labels = torch.tensor(self.data.iloc[idx]['label']).long()
            return image, labels
        else:
            return image
        
        def get_counter(self):
            print(f'{self.counter} observations were cropped with the sample from another class')

In [ ]:
####### EXAMINE SAMPLE BATCH

# sample size
sample = 5

# augmentations
train_augs, test_augs = get_augs(CFG, image_size = 128)

# datasets
train_dataset = LeafData(data         = df.head(sample*2), 
                         directory    = CFG['data_path'] + 'train_images/',
                         transform    = train_augs,
                         labeled      = True, 
                         pairing      = CFG['pairing'],
                         pairing_prob = 0.3,
                         seed         = CFG['seed'])
test_dataset = LeafData(data       = df.head(sample*2), 
                        directory  = CFG['data_path'] + 'train_images/',
                        transform  = test_augs,
                        labeled    = True)

# data loaders
train_loader = DataLoader(dataset     = train_dataset, 
                          batch_size  = sample, 
                          shuffle       = False, 
                          num_workers = 0,
                          pin_memory  = True)
test_loader = DataLoader(dataset      = test_dataset, 
                         batch_size   = sample, 
                         shuffle        = False, 
                         num_workers  = 0,
                         pin_memory   = True)

# display train images
batch_time = time.time()
for batch_idx, (inputs, labels) in enumerate(train_loader):

    # apply cutmix augmentation
    if CFG['cutmix'][0] > 0:
        mix_decision = 0 #np.random.rand(1)
        if mix_decision < CFG['cutmix'][0]:
            inputs, _ = cutmix_fn(data   = inputs, 
                                  target = labels, 
                                  alpha  = cutmix[1])

    # feedback
    inputs_shape = inputs.shape
    load_time    = time.time() - batch_time
    pixel_values = [torch.min(inputs).item(), torch.mean(inputs).item(), torch.max(inputs).item()]

    # examples
    fig = plt.figure(figsize = (20, 8))
    for i in range(sample):
        ax = fig.add_subplot(2, sample, i + 1, xticks = [], yticks = [])     
        plt.imshow(inputs[i].numpy().transpose(1, 2, 0))
        ax.set_title('Label: {} [train]'.format(labels[i].numpy()), color = 'red')
    break

# display test images
batch_time = time.time()
for batch_idx, (inputs, labels) in enumerate(test_loader):

    # feedback
    print('- batch shape:  {} vs {}'.format(inputs_shape, inputs.shape))
    print('- loading time: {:.4f} vs {:.4f} seconds'.format(load_time, (time.time() - batch_time)))
    print('- pixel values: {:.2f} - {:.2f} - {:.2f} vs {:.2f} - {:.2f} - {:.2f}'.format(
        pixel_values[0], pixel_values[1], pixel_values[2],
        torch.min(inputs).item(), torch.mean(inputs).item(), torch.max(inputs).item()))

    # examples
    for i in range(sample):
        ax = fig.add_subplot(2, sample, sample + i + 1, xticks = [], yticks = [])     
        plt.imshow(inputs[i].numpy().transpose(1, 2, 0))
        ax.set_title('Label: {} [valid]'.format(labels[i].numpy()), color = 'green')
    plt.savefig(CFG['out_path'] + 'fig_sample.png')
    break
    
# clean up
del inputs, labels, batch_idx, train_loader, test_loader, train_dataset, test_dataset, train_augs, test_augs

# CROSS-VALIDATION

In [ ]:
####### CROSS-VALIDATION LOOP

##### INITIALIZATION

# timer
cv_start = time.time()

# clear memory
import gc
gc.collect()

# placeholders
oof = None
perf = pd.DataFrame(columns = ['fold', 'epoch', 'trn_loss', 'val_loss', 'val_acc'])
if not CFG['training']:
    perf = pd.read_csv(CFG['out_path'] + 'tab_performance.csv')

# amp settings
amp_autocast = suppress
if CFG['use_amp']:
    amp_autocast = torch.cuda.amp.autocast
    scaler       = torch.cuda.amp.GradScaler()       

# adjust epochs if needed
if not CFG['training']:
    CFG['num_epochs'] = 1
    CFG['fine_tune']   = False
if CFG['debug']:
    CFG['num_epochs'] = 2
    CFG['num_folds']  = 2
    CFG['fine_tune']   = 1
    
    
##### CROSS-VALIDAION

for fold in range(CFG['num_folds']):
    
    ### PERFORM MODELING
    
    # feedback
    print('-' * 55)
    print('FOLD {:d}/{:d}'.format(fold + 1, CFG['num_folds']))    
    print('-' * 55) 
    
    # prepare data
    trn_loader, val_loader, df_trn, df_val = get_data(df, fold, CFG,
                                                      df_2019    = df_2019 if CFG['data_2019'] else None,
                                                      df_pl      = df_pl if CFG['data_pl'] else None,
                                                      df_ext     = df_ext if CFG['data_ext'] else None,
                                                      df_no      = df_no,
                                                      list_dupl  = list_dupl if CFG['drop_dupl'] else [],
                                                      list_noise = list_noise if CFG['drop_noise'] else []) 

    # training and inference
    if CFG['device'] != 'TPU':
        trn_losses, val_losses, val_metrics, val_preds_best = run_fold(fold       = fold, 
                                                                       trn_loader = trn_loader, 
                                                                       val_loader = val_loader, 
                                                                       df         = df,
                                                                       df_2019    = df_2019 if CFG['data_2019'] else None,
                                                                       df_pl      = df_pl if CFG['data_pl'] else None,
                                                                       df_ext     = df_ext if CFG['data_ext'] else None,
                                                                       df_no      = df_no,
                                                                       list_dupl  = list_dupl if CFG['drop_dupl'] else [],
                                                                       list_noise = list_noise if CFG['drop_noise'] else [],
                                                                       df_trn     = df_trn, 
                                                                       df_val     = df_val, 
                                                                       CFG        = CFG, 
                                                                       device     = device)
    else:
        def _mp_fn(rank, CFG):
            torch.set_default_tensor_type('torch.FloatTensor')
            device = xm.xla_device()
            trn_losses, val_losses, val_metrics, val_preds_best = run_fold(fold       = fold, 
                                                                            trn_loader = trn_loader, 
                                                                            val_loader = val_loader, 
                                                                            df         = df,
                                                                            df_2019    = df_2019 if CFG['data_2019'] else None,
                                                                            df_pl      = df_pl if CFG['data_pl'] else None,
                                                                            df_ext     = df_ext if CFG['data_ext'] else None,
                                                                            df_no      = df_no,
                                                                            list_dupl  = list_dupl if CFG['drop_dupl'] else [],
                                                                            list_noise = list_noise if CFG['drop_noise'] else [],
                                                                            df_trn     = df_trn, 
                                                                            df_val     = df_val, 
                                                                            CFG        = CFG, 
                                                                            device     = device)
            if rank == 0:
                np.save('trn_losses.npy',     np.array(trn_losses))
                np.save('val_losses.npy',     np.array(val_losses))
                np.save('val_metrics.npy',    np.array(val_metrics))
                np.save('val_preds_best.npy', val_preds_best)
        xmp.spawn(_mp_fn, args = (CFG, ), nprocs = CFG['num_workers'], start_method = 'fork')
        trn_losses          = np.load('trn_losses.npy')
        val_losses          = np.load('val_losses.npy')
        val_metrics         = np.load('val_metrics.npy')
        val_preds_best      = np.load('val_preds_best.npy')


    ### SAVE RESULTS
    
    # performance table
    if CFG['training']:
        perf = pd.concat([perf, 
                          pd.DataFrame({'fold':     [fold] * (CFG['num_epochs'] + CFG['fine_tune']), 
                                        'epoch':    list(range(CFG['num_epochs'] + CFG['fine_tune'])),
                                        'trn_loss': trn_losses,
                                        'val_loss': val_losses,
                                        'val_acc':  val_metrics})],
                        axis = 0)
        perf.to_csv(CFG['out_path'] + 'tab_performance.csv', index = False)
            
    # export OOF predictions
    val_preds_df = pd.DataFrame(val_preds_best, columns = ['c' + str(class_idx) for class_idx in range(CFG['num_classes'])])
    val_preds_df = pd.concat([df_val, val_preds_df], axis = 1)
    oof = pd.concat([oof, val_preds_df], axis = 0).reset_index(drop = True)
    oof.to_csv(CFG['out_path'] + 'oof.csv', index = False)


    ### DISPLAY FEEDBACK

    # feedback
    print('-' * 55)
    print('Best: acc = {:.4f} (epoch {}), loss = {:.4f} (epoch {})'.format(
        np.max(val_metrics), np.argmax(val_metrics) + 1, np.min(val_losses), np.argmin(val_losses) + 1))
    print('-' * 55)

    # plot loss dynamics
    if CFG['training']:
        plot_results(trn_losses, val_losses, val_metrics, fold, CFG)


##### FEEDBACK

print('')
print('Finished in {:.2f} minutes'.format((time.time() - cv_start) / 60))

# EVALUATION

In [ ]:
####### CHECK OOF PERFORMANCE

# compute accuracy
oof['pred'] = np.argmax(oof[['c' + str(class_idx) for class_idx in range(CFG['num_classes'])]].values, axis = 1)
oof_acc     = (oof['pred'] == oof['label']).sum() / len(oof)
oof_loss    = perf.groupby('fold')['val_loss'].agg('min').mean()

# print results
print('- OOF acc  = {:.4f}'.format(oof_acc))
print('- OOF loss = {:.4f}'.format(oof_loss))

# save results
res = pd.DataFrame({'fold':     ['oof',], 
                    'epoch':    ['best'],
                    'trn_loss': [np.nan],
                    'val_loss': [oof_loss],
                    'val_acc':  [oof_acc]})
perf = pd.concat([perf, res], axis = 0)
perf.to_csv(CFG['out_path'] + 'tab_performance.csv', index = False)

In [ ]:
####### CONFUSION MATRIX

# construct confusion matrx
cm    = confusion_matrix(y_true = oof['label'], y_pred = oof['pred'], normalize = 'all')
annot = np.around(cm, 2)

# plot matrix
fig, ax = plt.subplots(figsize = (10, 8))
sns.heatmap(cm, cmap = 'Blues', annot = annot, lw = 0.5)
ax.set_xlabel('Prediction')
ax.set_ylabel('Ground Truth')
ax.set_aspect('equal')
    
# export plot
plt.savefig(CFG['out_path'] + 'fig_confusion.png')

In [ ]:
####### EXPORT CONFIGURATION

# store performance values
CFG['time_min'] = (time.time() - cv_start) / 60
CFG['oof_acc']  = oof_acc
CFG['oof_loss'] = oof_loss

# save DF
params = pd.DataFrame.from_dict(CFG, orient = 'index', columns = ['value'])
params.to_csv(CFG['out_path'] + 'tab_configuration.csv', index = True)